<a href="https://colab.research.google.com/github/zdwhite/Thinkful-Unit-2/blob/master/Cross_validation_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Grab and process the raw data.
data_path = ("https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/"
             "master/sms_spam_collection/SMSSpamCollection"
            )
sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['spam', 'message']

# Enumerate our spammy keywords.
keywords = ['click', 'offer', 'winner', 'buy', 'free', 'cash', 'urgent']

for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

sms_raw['allcaps'] = sms_raw.message.str.isupper()
sms_raw['spam'] = (sms_raw['spam'] == 'spam')
data = sms_raw[keywords + ['allcaps']]
target = sms_raw['spam']

from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

In [0]:
target

In [0]:
# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

With 20% Holdout: 0.884304932735426
Testing on Sample: 0.8916008614501076


In [0]:
from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

array([0.89784946, 0.89426523, 0.89426523, 0.890681  , 0.89605735,
       0.89048474, 0.88150808, 0.89028777, 0.88489209, 0.89568345])

In [0]:
# Implement your own cross validation with your spam model.

#define a function that takes in 4 variables
# model type
# data set
# answer key
# number of  "folds"

#Returns
#an array of accuracy equal in length to the number of folds

#dir(sklearn.naive_bayes)

def cross_val(model, data, folds):
  import sklearn.naive_bayes
  accuracy = []
  
  sample_size = round((len(data)/folds))
 
  test_fold=data
  
  while len(test_fold) > sample_size :
  #create sample to model
    test_sample = test_fold.sample(sample_size)
  #drop the sample from the set so it can't be resampled
    test_fold=test_fold.drop(test_sample.index)
    
    test_data=test_sample[keywords + ['allcaps']]
    test_target=test_sample['spam']
  
    score = model.fit(test_data,test_target).score(test_data,test_target)
    accuracy.append(score)
  return accuracy
  #make n samples of the data variable
  ## Where n = folds
  #run each fold through the given model
  #aggregate results and return an array
  
  

In [0]:

cross_val(bnb,sms_raw, folds=10)

[0.9066427289048474,
 0.8689407540394973,
 0.9102333931777379,
 0.8797127468581688,
 0.9048473967684022,
 0.8761220825852782,
 0.9066427289048474,
 0.8850987432675045,
 0.9012567324955116,
 0.8868940754039497]